# Visualising MCC Exploration

This notebook logs exploratory results on adding teleportation on MCC with state coverage visualisation

21/01/2024
- Naive teleportation to argmax works
- Longer episodes are better than shorter
- Different intrinsic rewards show significantly different behavior
- Even naively, general improvement over pure intrinsic
- Fails to beat intrinsic + extrinsic: perhaps this is due to negative extrinsic reward revealing data on target? Not comparable, and I think fully explored in that reward shifting paper
- Keeps teleporting to same target
- This may be a problem with DDPG


28/01/2024
- Probabilistic teleportation work well
- Environment reset stochasticity is important
- Time limit aware Q functions are difficult to train!
- Proposal: Dynamic Truncation!

4/02/2024
- ICM and RND leads to inherently different results - RND should be prioritised
- CATS fails to improve over baseline on RND with fixed reset, but does in ICM. After reset, the new trajectory follows the previous trajectory too closely, while resetting from the start leads to more divergence across the entire episode (and hence more exploration)
- Fixing the reset states leads to improved analysis
- Policy function gets stuck in the local minima of the Q function
- Analyse DQN instead? Skip parametrized policy function and use an approximator?? Maybe implement QT-opt https://arxiv.org/pdf/1806.10293.pdf. This may be important to obtain interesting experiment results, since on MCC the policy generally fails to follow the critic even on large learning rates (why??)

11/02/2024
- Ensemble bootstrapping (Thompson sampling) seems to have uncertain impact over baseline, maybe slightly positive?


TODO:
- Confidence Bounds (How? Without latent density estimator?)
- Termination as an action
- Epsilon greedy
- Time aware exploration

Known Failure Modes
- Teleporting to the end of the episode, and immediately truncating
- 

Ideas
- Bootstrapped Q value estimate for confidence bound guided estimation?

Interesting observations
- Qt_opt directly on critic, rather than target network explores faster??

In [ ]:
# Define Imports and shared training information

# Std
import copy
import math
from typing import Optional

# Training
import numpy as np
import torch
from tqdm import tqdm
from omegaconf import DictConfig
import gymnasium as gym

# Evaluation
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.cm as cm

# Curiosity
from curiosity.experience import Transition
from curiosity.experience.collector import GymCollector
from curiosity.policy import ColoredNoisePolicy, Policy
from curiosity.experience.memory import ReplayBuffer
from curiosity.experience.util import build_replay_buffer
from curiosity.util.util import *
from curiosity.rl.qt_opt import cross_entropy_method

from cats import CatsExperiment
from evaluation import entropy_memory

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
COLLECTION_STEPS = 4096
MAX_EPISODE_STEPS = 400
ENV = "MountainCarContinuous-v0"
#ENV = "Pendulum-v1"

cfg = DictConfig({
    "env": {
        "name": ENV,
    },
    "memory": {
        "type": "experience_replay",
    #    "type": "prioritized_experience_replay",
    #    "alpha": 0.6,
    #    "epsilon": 0.1,
    #    "beta_0": 0.4,
    #    "beta_annealing_steps": COLLECTION_STEPS,
        "normalise_observation": True
    },
    "train": {
        "initial_collection_size": 256,
        "total_frames": COLLECTION_STEPS,
        "minibatch_size": 128
    },
    "algorithm": {
        "type": "qt_opt",
        "gamma": 0.99,
        "tau": 0.005,
        "lr": 0.01,
        "update_frequency": 1,
        "clip_grad_norm": 1,
        "ensemble_number": 5,
        "actor": {
            "features": 128
        },
        "critic": {
            "features": 128
        }
    },
    "intrinsic": {
        "type": "rnd",
        "encoding_size": 32,
        "lr": 0.0003,
        "int_coef": 1, 
        "ext_coef": 2,
        "reward_normalisation": True,
        "normalised_obs_clip": 5
    },
    "noise": {
        "scale": 0.1,
        "beta": 0
    }
})


In [ ]:
experiment = CatsExperiment(
    cfg=cfg,
    collection_steps=COLLECTION_STEPS,
    device=DEVICE,
    enable_policy_sampling=True,
    teleport_strategy=("e_greedy", 0.1),
    seed=7
)
experiment.run()

In [ ]:
print(experiment.log['total_intrinsic_reward'])
print(entropy_memory(experiment.memory))

In [ ]:
experiment = CatsExperiment(
    cfg=cfg,
    collection_steps=COLLECTION_STEPS,
    device=DEVICE,
    enable_policy_sampling=True,
    teleport_strategy=("thompson", 2),
    seed=7
)
experiment.run()

In [ ]:
print(experiment.log['total_intrinsic_reward'])
print(entropy_memory(experiment.memory))

In [ ]:
experiment = CatsExperiment(
    cfg=cfg,
    collection_steps=COLLECTION_STEPS,
    device=DEVICE,
    enable_policy_sampling=True,
    teleport_strategy=None,
    seed=7
)
experiment.run()

In [ ]:
print(experiment.log['total_intrinsic_reward'])
print(entropy_memory(experiment.memory))

# Bootstrapping Intrinsic Motivation

Does it help?

In [ ]:
N = 10

baseline_statistics = {
    'total_intrinsic_reward': [],
    'newly_collected_intrinsic_reward': [],
    'entropy': []
}
bootstrapping_statistics = copy.deepcopy(baseline_statistics)

# Baselne
for seed in range(0, N):
    experiment = CatsExperiment(
        cfg=cfg,
        collection_steps=COLLECTION_STEPS,
        seed=seed,
        device=DEVICE,
        enable_policy_sampling=False
    )
    experiment.run()
    for k,v in experiment.log.items():
        baseline_statistics[k].append(v)
    baseline_statistics['entropy'].append(entropy_memory(experiment.memory))

for seed in range(0, N):
    experiment = CatsExperiment(
        cfg=cfg,
        collection_steps=COLLECTION_STEPS,
        seed=seed,
        device=DEVICE,
        enable_policy_sampling=True
    )
    experiment.run()
    for k,v in experiment.log.items():
        bootstrapping_statistics[k].append(v)
    bootstrapping_statistics['entropy'].append(entropy_memory(experiment.memory))


In [ ]:
print("Baseline")
print(f"Total Intrinsic Reward: {sum(baseline_statistics['total_intrinsic_reward'])/N}")
print(f"Collected Intrinsic Reward: {sum(baseline_statistics['newly_collected_intrinsic_reward'])/N}")
print(f"Entropy: {sum(baseline_statistics['entropy'])/N}")
print("Bootstrapped")
print(f"Total Intrinsic Reward: {sum(bootstrapping_statistics['total_intrinsic_reward'])/N}")
print(f"Collected Intrinsic Reward: {sum(bootstrapping_statistics['newly_collected_intrinsic_reward'])/N}")
print(f"Entropy: {sum(bootstrapping_statistics['entropy'])/N}")

In [ ]:
experiment.log

In [ ]:


def visualise_memory(env: gym.Env, *memories: ReplayBuffer):
    """ Visualise state space for given environmentss
    """
    
    fig, ax = plt.subplots()
    ax.set_title("State Space Coverage")

    if env.spec.id == "MountainCarContinuous-v0":
        ax.set_xlim(env.observation_space.low[0], env.observation_space.high[0])
        ax.set_xlabel("Position")
        ax.set_ylim(env.observation_space.low[1], env.observation_space.high[1])
        ax.set_ylabel("Velocity")
    elif env.spec.id == "Pendulum-v1":
        ax.set_xlim(-math.pi, math.pi)
        ax.set_xlabel("Theta")
        ax.set_ylim(env.observation_space.low[2], env.observation_space.high[2])
        ax.set_ylabel("Angular Velocity")

    for memory, name in memories:
        batch = Transition(*memory.storage)
        s = batch.s_0.cpu().numpy()

        # Colors based on time
        norm = mpl.colors.Normalize(vmin=0, vmax=len(s)-1)
        cmap = cm.viridis
        m = cm.ScalarMappable(norm=norm, cmap=cmap)

        colors = m.to_rgba(np.linspace(0, len(s)-1, len(s)))

        if env.spec.id == "MountainCarContinuous-v0":
            ax.scatter(s[:, 0], s[:, 1], s=1, label=name, c=colors)
        elif env.spec.id == "Pendulum-v1":
            ax.scatter(np.arctan2(s[:, 1], s[:, 0]), s[:, 2], s=1, label=name, c=colors)

    
    fig.colorbar(m, ax=ax)
    ax.legend()

# Visualise V
# For mountain car

def visualise_experiment_v(experiment):

    X = torch.linspace(experiment.env.observation_space.low[0], experiment.env.observation_space.high[0], 100)
    Y = torch.linspace(experiment.env.observation_space.low[1], experiment.env.observation_space.high[1], 100)
    grid_X, grid_Y = torch.meshgrid((X, Y))

    # Time Aware
    #states = torch.stack((grid_X.flatten(), grid_Y.flatten(), 1+torch.zeros_like(grid_X.flatten())) ).T

    fig, axs = plt.subplots(1,3)
    fig.set_size_inches(20,5)
    ax = axs[0]
    states = torch.stack((grid_X.flatten(), grid_Y.flatten())).T
    states = states.to(DEVICE)
        # Observation Normalisation
    states_cpu = states.cpu()
    states = (states - experiment.policy.normalise_obs.mean) / experiment.policy.normalise_obs.std

    # V 
    values = experiment.V(states)
    norm = mpl.colors.Normalize(vmin=values.min(), vmax=values.max())
    cmap = cm.viridis
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    colors = m.to_rgba(values.detach().cpu())

    ax.set_title("Value Function Visualisation")
    ax.scatter(states_cpu[:,0], states_cpu[:, 1], c=colors)
    fig.colorbar(m, ax=ax)

    # Policy

    # Train new actor
    #
    # new_actor = build_actor(experiment.env, cfg.algorithm.actor.features).to(DEVICE)
    # optim_actor = torch.optim.Adam(params=new_actor.net.parameters(), lr=0.1)
    # X = torch.linspace(experiment.env.observation_space.low[0], experiment.env.observation_space.high[0], 10)
    # Y = torch.linspace(experiment.env.observation_space.low[1], experiment.env.observation_space.high[1], 10)
    # grid_X, grid_Y = torch.meshgrid((X, Y))
    # pseudo_states = torch.stack((grid_X.flatten(), grid_Y.flatten())).to(DEVICE)
    # pseudo_states = pseudo_states.T
    # pseudo_states = (pseudo_states - experiment.policy.normalise_obs.mean) / experiment.policy.normalise_obs.std
# 
    # for _ in range(200):
    #     # batch, aux = experiment.memory.sample(16)
    #     # batch = Transition(*batch)
    #     # s_0 = batch.s_0
    #     s_0 = pseudo_states
    #     desired_action = new_actor(s_0)
    #     loss = -experiment.algorithm.critic(torch.cat((s_0, desired_action), 1))
    #     loss = torch.mean(torch.mean(loss, dim=list(range(1, len(loss.shape)))))
    #     optim_actor.zero_grad()
    #     loss.backward()
    #     torch.nn.utils.clip_grad_norm_(new_actor.net.parameters(), 1)
    #     optim_actor.step()
# 
    #     # Evaluate differences between old and new actor
    # batch, aux = experiment.memory.sample(1024)
    # print("Experiment policy value", experiment.V(batch[0]).mean().detach().item())
# 
    # target_action = new_actor(batch[0])
    # V = experiment.algorithm.critic.target(torch.cat((batch[0], target_action), 1))
    # print("Retrained policy value ", V.mean().detach().item())
# 


    ax = axs[1]
    actions = experiment.algorithm.policy_fn(states)
    #actions = new_actor(states)
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    cmap = cm.viridis
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    colors = m.to_rgba(actions.detach().cpu())
    ax.scatter(states_cpu[:,0], states_cpu[:, 1], c=colors)
    ax.set_title("Policy Actions")
    fig.colorbar(m, ax=ax)

    # Q Function Optimal
    ax = axs[2]

    batch_size = 1000
    actions = []
    for i in range(100*100 // batch_size): 
        batch = states[i*batch_size : (i+1)*batch_size]
        a=cross_entropy_method(
                batch,
                experiment.algorithm.critic,
                experiment.collector.env.action_space,
                device=DEVICE
            )
        actions.append(a)

    actions = torch.cat(actions)
    print(actions.shape)
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    cmap = cm.viridis
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    colors = m.to_rgba(actions.detach().cpu())
    ax.scatter(states_cpu[:,0], states_cpu[:, 1], c=colors)
    ax.set_title("Cross Entropy Maximal Actions")
    fig.colorbar(m, ax=ax)

    print(f"Minimum Value {values.min()} | Maximum Value {values.max()}")

In [ ]:
# Intrinsic Only

    # Build RL Structure
experiment = Experiment(
    cfg,
    max_episode_steps=MAX_EPISODE_STEPS,
    death_is_not_the_end=True,
    fixed_reset=True
)
experiment.run()

    # Visualise Training Information
visualise_memory(
     experiment.env,
    (experiment.memory, "intrinsic"),
)

In [ ]:
visualise_memory(
     experiment.env,
    (experiment.memory, "intrinsic"),
)

In [ ]:
visualise_experiment_v(experiment)
print("Entropy: ", entropy_memory(experiment.memory))

In [ ]:
visualise_experiment_v(experiment)
print("Entropy: ", entropy_memory(experiment.memory))

In [ ]:
class CATS(Experiment):

    def __init__(self,
                 cfg,
                 max_episode_steps: Optional[int] = None,
                 death_is_not_the_end: bool = True,
                 fixed_reset: bool = False,
                 epsilon: float = 0.2):
        super().__init__(cfg, max_episode_steps, death_is_not_the_end, fixed_reset)

        # Recently explored trajectory
        self.trajectory = torch.zeros((self.env.spec.max_episode_steps, *self.env.observation_space.shape), device=DEVICE)
        # Current time step
        self.trajectory_index = 0
        # Target Timestep
        self.teleport_index = 0
        # Reset epsilon
        self.epsilon = epsilon
        
        # Environment deepcopies
        self.state = None
        self.quicksaves = [None for _ in range(self.env.spec.max_episode_steps)]

        # RNG
        self.np_rng = np.random.default_rng(self.cfg.seed)

        self.log["teleport_targets"] = []
        self.log["teleport_targets_observations"] = []
        self.log["latest_trajectory"] = []

    def _build_env(self, max_episode_steps=None):
        super()._build_env(max_episode_steps)
        
        # Meta-RL 

            # Wrap Time Aware Observations
            # This adds complexity to the environments, hard to learn?
        # self.env = TimeAwareObservation(self.env)
        #    # Adjust observation space limit
        # igh = self.env.observation_space.high
        # igh[-1] = self.env.spec.max_episode_steps
        # elf.env.observation_space = gym.spaces.Box(
        #    self.env.observation_space.low,
        #    high,

            # Learn to Truncate
        # self.env = self.env.env

    def _teleport_selection(self, V):
        V = V**2
            # Argmax
        #teleport_index = torch.argmax(V).item()
            # Probability Matching
        p = V / V.sum()
        pt = self.np_rng.random()
        pc = 0
        for i, pi in enumerate(p):
            pc += pi
            if pc >= pt or i == len(p) - 1:
                teleport_index = i
                break
            # TODO: Upper Confidence bound
        return teleport_index
        
    def _reset(self, V):
        if self.fixed_reset:
            return
            # Epsilon Greedy Reset
        #if torch.rand(1) < epsilon:
        #    obs, infos = collector.env.reset()
        #    resets[0] = collector.env
        #    trajectory[0] = torch.tensor(obs, device=DEVICE)
        #    teleport_index = 0
            # Reset Buffer
        reset_buffer = []
        reset_buffer_obs = []
        for i in range(10):
            obs, info = self.collector.env.reset(seed=int(self.np_rng.integers(65536)))
            reset_buffer.append(copy.deepcopy(self.collector.env))
            reset_buffer_obs.append(obs)
        reset_buffer_obs = torch.tensor(np.array(reset_buffer_obs, dtype=np.float32), device=DEVICE)
        V_r = self.V(reset_buffer_obs)
        best_reset_index = torch.argmax(V_r).item()
        #if self.np_rng.random() < self.epsilon or V_r[best_reset_index] >= V[self.teleport_index]:
        if V_r[best_reset_index] >= V[self.teleport_index]:
            self.collector.env = reset_buffer[best_reset_index]
            self.quicksaves[0] = self.collector.env
            self.trajectory[0] = reset_buffer_obs[best_reset_index]
            self.teleport_index = 0

    def run(self):
        self.early_start(cfg.train.initial_collection_size)
        self.env_reset()

        batch, aux = self.memory.sample(self.cfg.train.initial_collection_size)
        self.intrinsic.initialise(Transition(*batch), aux)

        # Main Loop
        self.state = copy.deepcopy(self.collector.env)
        self.newly_collected_intrinsic_reward = []
        for step in tqdm(range(1, cfg.train.total_frames+1)):
            obs, action, reward, n_obs, terminated, truncated = self.collector.collect(n=1)[-1]

            self.newly_collected_intrinsic_reward.append(self.intrinsic.reward(Transition(
                torch.tensor(obs, device=DEVICE).unsqueeze(0),
                torch.tensor(action, device=DEVICE).unsqueeze(0),
                torch.tensor(reward, device=DEVICE).unsqueeze(0),
                torch.tensor(n_obs, device=DEVICE).unsqueeze(0),
                torch.tensor(terminated, device=DEVICE).unsqueeze(0)
            ))[2].item())

            # Time Limit Normalisation
            # obs, n_obs = copy.deepcopy(obs), copy.deepcopy(n_obs)
            # obs[-1] = obs[-1] / self.env.spec.max_episode_steps
            # n_obs[-1] = n_obs[-1] / self.env.spec.max_episode_steps

            # Update trajectory
            self.quicksaves[self.trajectory_index] = self.state
            self.trajectory[self.trajectory_index] = torch.from_numpy(obs).to(DEVICE)
            self.state = copy.deepcopy(self.collector.env)
            self.trajectory_index += 1

            # Manage Teleportation
            if truncated or terminated:
                # Calculate Value
                V = self.V(self.trajectory[:self.trajectory_index])
                # Teleportation Selection
                with torch.no_grad():
                    self.teleport_index = self._teleport_selection(V)
                #print(self.teleport_index)
                # Resets
                self._reset(V)

                # Teleport
                self.collector.env = self.quicksaves[self.teleport_index]
                self.collector.obs = self.trajectory[self.teleport_index].cpu().numpy()
                self.collector.env.np_random = np.random.default_rng(self.np_rng.integers(65536))
                self.trajectory_index = self.teleport_index
                self.state = copy.deepcopy(self.collector.env)

                # Log
                self.log["teleport_targets"].append(self.teleport_index)
                self.log["teleport_targets_observations"].append(self.collector.obs)
                self.log["latest_trajectory"] = self.trajectory.cpu().numpy()

                # Account for time
                # terminated = True
                # n_obs = self.collector.obs


            # Update Memory
            self._update_memory(obs, action, reward, n_obs, terminated, truncated)
            
            # Remaining RL Update
            batch, aux = self.memory.sample(self.cfg.train.minibatch_size)
            batch = Transition(*batch)
            if self.death_is_not_the_end:
                batch = Transition(batch.s_0, batch.a, batch.r, batch.s_1, torch.zeros(batch.d.shape, device=DEVICE).bool())


            r_t, r_e, r_i = self.intrinsic.reward(batch)
            r_i = r_i
            self.intrinsic.update(batch, aux, step=step)

            batch = Transition(batch.s_0, batch.a, r_i, batch.s_1, batch.d)
            self.algorithm.update(batch, aux, step=step)

            # Log
            self.log["total_intrinsic_reward"] += r_i.mean().item()
        

In [ ]:
cats = CATS(
    cfg,
    max_episode_steps=MAX_EPISODE_STEPS,
    death_is_not_the_end=True,
    fixed_reset=True
)
cats.run()

    # Visualise Training Information
visualise_memory(
      cats.env,
     (cats.memory, "teleport"),
)

In [ ]:
plt.scatter(
    cats.log['latest_trajectory'][:,0],
    cats.log['latest_trajectory'][:,1]
)

In [ ]:
plt.plot(cats.log['teleport_targets'])

In [ ]:
visualise_experiment_v(cats)
print("Entropy: ", entropy_memory(cats.memory))

In [ ]:
# Learn To Truncate

class CATS_Truncate(Experiment):

    def __init__(self,
                 cfg,
                 max_episode_steps: Optional[int] = None,
                 death_is_not_the_end: bool = True,
                 fixed_reset: bool = False,
                 storage_size: float = 100,
                 epsilon: float = 0.1):
        super().__init__(cfg, max_episode_steps, death_is_not_the_end, fixed_reset)

        self.storage_size = storage_size
        # Recently explored trajectory
        self.trajectory = []

        # Current time step
        self.trajectory_index = 0
        self.trajectory_counter = 0
        # Target Timestep
        self.teleport_index = 0
        # Reset epsilon
        self.epsilon = epsilon
        
        # Environment deepcopies
        self.state = None
        self.quicksaves = []

        # RNG
        self.np_rng = np.random.default_rng(self.cfg.seed)

        self.log["teleport_targets"] = []
        self.log["teleport_targets_observations"] = []
        self.log["latest_trajectory"] = []

    def _build_env(self, max_episode_steps=None):
        super()._build_env(max_episode_steps)

        # Learn to Truncate
        self.env = self.env.env

    def _teleport_selection(self, V):
        #V = V**2
            # Argmax
        teleport_index = torch.argmax(V).item()
            # Probability Matching
        #p = V / V.sum()
        # pt = self.np_rng.random()
        # pc = 0
        # for i, pi in enumerate(p):
        #     pc += pi
        #     if pc >= pt or i == len(p) - 1:
        #         teleport_index = i
        #         break
        #     # TODO: Upper Confidence bound
        return teleport_index

    def _reset(self, V):
        if self.fixed_reset:
            return
            # Epsilon Greedy Reset
        #if torch.rand(1) < epsilon:
        #    obs, infos = collector.env.reset()
        #    resets[0] = collector.env
        #    trajectory[0] = torch.tensor(obs, device=DEVICE)
        #    teleport_index = 0
            # Reset Buffer
        reset_buffer = []
        reset_buffer_obs = []
        for i in range(10):
            obs, info = self.collector.env.reset(seed=int(self.np_rng.integers(65536)))
            reset_buffer.append(copy.deepcopy(self.collector.env))
            reset_buffer_obs.append(obs)
        reset_buffer_obs = torch.tensor(np.array(reset_buffer_obs, dtype=np.float32), device=DEVICE)
        V_r = self.V(reset_buffer_obs)
        best_reset_index = torch.argmax(V_r).item()
        #if self.np_rng.random() < self.epsilon or V_r[best_reset_index] >= V[self.teleport_index]:
        if V_r[best_reset_index] >= V[self.teleport_index]:
            self.collector.env = reset_buffer[best_reset_index]
            self.quicksaves[0] = self.collector.env
            self.trajectory[0] = reset_buffer_obs[best_reset_index]
            self.teleport_index = 0

    def run(self):
        if self.fixed_reset:
            self.collector.env.reset(seed=self.cfg.seed)

        early_start_transitions = self.collector.early_start(cfg.train.initial_collection_size)
        for t in early_start_transitions:
            self._update_memory(*t)

        batch, aux = self.memory.sample(self.cfg.train.initial_collection_size)
        self.intrinsic.initialise(Transition(*batch), aux)

        # Main Loop
        if self.fixed_reset:
            self.collector.env.reset(seed=self.cfg.seed)
        else:
            self.collector.env.reset()

        self.state = copy.deepcopy(self.collector.env)
        newly_collected_intrinsic_reward = 0
        for step in tqdm(range(1, cfg.train.total_frames+1)):
            obs, action, reward, n_obs, terminated, _ = self.collector.collect(n=1)[-1]

            newly_collected_intrinsic_reward += self.intrinsic.reward(Transition(
                torch.tensor(obs, device=DEVICE).unsqueeze(0),
                torch.tensor(action, device=DEVICE).unsqueeze(0),
                torch.tensor(reward, device=DEVICE).unsqueeze(0),
                torch.tensor(n_obs, device=DEVICE).unsqueeze(0),
                torch.tensor(terminated, device=DEVICE).unsqueeze(0)
            ))[2].item()

            if self.trajectory_counter == self.storage_size - 1:
                truncated = True
            else:
                truncated = False
            # Time Limit Normalisation
            # obs, n_obs = copy.deepcopy(obs), copy.deepcopy(n_obs)
            # obs[-1] = obs[-1] / self.env.spec.max_episode_steps
            # n_obs[-1] = n_obs[-1] / self.env.spec.max_episode_steps

            # Update trajectory
            self.quicksaves.append(self.state)
            self.trajectory.append(obs)
            self.state = copy.deepcopy(self.collector.env)
            self.trajectory_index += 1
            self.trajectory_counter += 1

            # Manage Teleportation
            if truncated or terminated:
                # Calculate Value
                V = self.V(torch.tensor(self.trajectory,device=DEVICE))
                # Teleportation Selection
                with torch.no_grad():
                    self.teleport_index = self._teleport_selection(V)
                # Resets
                self._reset(V)

                # Teleport
                self.collector.env = self.quicksaves[self.teleport_index]
                self.collector.obs = self.trajectory[self.teleport_index]
                self.collector.env.np_random = np.random.default_rng(self.np_rng.integers(65536))

                self.trajectory_index = self.teleport_index
                self.trajectory_counter = 0
                self.trajectory = self.trajectory[:self.teleport_index+1]
                self.quicksaves = self.quicksaves[:self.teleport_index+1]
                self.state = copy.deepcopy(self.collector.env)

                # Log
                self.log["teleport_targets"].append(self.teleport_index)
                self.log["teleport_targets_observations"].append(self.collector.obs)
                self.log["latest_trajectory"] = self.trajectory

            # Update Memory
            self._update_memory(obs, action, reward, n_obs, terminated, truncated)
            
            # Remaining RL Update
            batch, aux = self.memory.sample(self.cfg.train.minibatch_size)
            batch = Transition(*batch)
            if self.death_is_not_the_end:
                batch = Transition(batch.s_0, batch.a, batch.r, batch.s_1, torch.zeros(batch.d.shape, device=DEVICE).bool())


            r_t, r_e, r_i = self.intrinsic.reward(batch)
            self.intrinsic.update(batch, aux, step=step)

            batch = Transition(batch.s_0, batch.a, r_i, batch.s_1, batch.d)
            self.algorithm.update(batch, aux, step=step)

            # Log
            self.log["total_intrinsic_reward"] += r_i.mean().item()

        print(newly_collected_intrinsic_reward)
        

In [ ]:
cats_truncate = CATS_Truncate(
    cfg,
    max_episode_steps=9999,
    death_is_not_the_end=True,
    storage_size=400,
    fixed_reset=True
)
cats_truncate.run()

    # Visualise Training Information
visualise_memory(
      cats_truncate.env,
     (cats_truncate.memory, "teleport"),
)

In [ ]:
visualise_experiment_v(cats_truncate)
print("Entropy: ", entropy_memory(cats_truncate.memory))

In [ ]:
print(cats.log['total_intrinsic_reward'])
print(experiment.log['total_intrinsic_reward'])
print(cats_truncate.log['total_intrinsic_reward'])

In [ ]:
fig, ax = plt.subplots()
ax.plot(experiment.newly_collected_intrinsic_reward[MAX_EPISODE_STEPS:], label="baseline")
ax.plot(cats.newly_collected_intrinsic_reward[MAX_EPISODE_STEPS:], label="cats")
ax.plot(cats_truncate.newly_collected_intrinsic_reward[MAX_EPISODE_STEPS:], label="cats_truncate")
fig.legend()